# Variational Inference with Planar Flows for Implied Volatility Surfaces

In this notebook, we will implement a variational autoencoder (VAE) with planar flows to model and fill implied volatility (IV) surfaces. We will use PyTorch to build and train the model. The key components include:
1. Generating synthetic sample data for IV surfaces and features.
2. Defining the Planar Flow layer to ensure invertibility.
3. Creating the Encoder and Decoder networks.
4. Implementing the VAE with planar flows.
5. Training the model and evaluating its performance.

## Imports

First, we import the necessary libraries.


In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn

## Sample Data Generation

We generate synthetic sample data for implied volatility (IV) surfaces and additional features. The IV surfaces are represented as grids with some missing values to simulate real-world scenarios.

- `iv_grids`: A 10x10 grid of IV values.
- `features`: Additional asset and market features.
- `strike_times`: Strike prices and times to maturity for the options.

We flatten the IV grids and concatenate them with the features and strike_times for the encoder input. The target IV points are the center points of the grids.


In [ ]:
# Generate synthetic sample data
num_samples = 1000
iv_grid_dim = (10, 10)  # 10x10 IV grid
feature_dim = 5  # Number of asset and market features

# Create random IV grids with some NaNs to simulate missing data
iv_grids = np.random.rand(num_samples, *iv_grid_dim)
iv_grids[np.random.rand(*iv_grids.shape) < 0.1] = np.nan  # Introduce some NaNs

# Create random asset and market features
features = np.random.rand(num_samples, feature_dim)

# Create random spot prices, strike prices, and times to maturity in days
spot_prices = np.random.rand(num_samples) * 100  # Example spot prices
strike_prices = np.random.rand(num_samples) * 100  # Example strike prices
time_to_maturity_days = np.random.randint(1, 365, num_samples)

# Compute log moneyness and time to maturity in years
log_moneyness = np.log(spot_prices / strike_prices)
time_to_maturity_years = time_to_maturity_days / 365

# Flatten the IV grids and concatenate with features for the encoder input
iv_grids_flat = iv_grids.reshape(num_samples, -1)
encoder_inputs = np.hstack((iv_grids_flat, features))

# Prepare the target data and conditional features for the decoder
targets = []
cond_features = []

for i in range(num_samples):
    for row in range(iv_grid_dim[0]):
        for col in range(iv_grid_dim[1]):
            if not np.isnan(iv_grids[i, row, col]):
                targets.append(iv_grids[i, row, col])
                cond_features.append(np.hstack((features[i], log_moneyness[i], time_to_maturity_years[i])))

targets = np.array(targets)
cond_features = np.array(cond_features)

# Convert to PyTorch tensors
encoder_inputs = torch.tensor(encoder_inputs, dtype=torch.float32)
features_tensor = torch.tensor(features, dtype=torch.float32)
targets_tensor = torch.tensor(targets, dtype=torch.float32).unsqueeze(1)  # Make targets (num_targets, 1)
cond_features_tensor = torch.tensor(cond_features, dtype=torch.float32)

# Ensure encoder inputs are repeated for each target
encoder_inputs_repeated = encoder_inputs.repeat_interleave(len(targets_tensor), dim=0)

# Create data loader
batch_size = 32
dataset = TensorDataset(encoder_inputs_repeated, targets_tensor)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Create loader for conditioning features (features, log moneyness, and time to maturity)
cond_features_loader = DataLoader(cond_features_tensor, batch_size=batch-size, shuffle=True)

## Planar Flow Layer

We define the Planar Flow layer which modifies its parameters to ensure invertibility. This involves using the condition \( w^T u \geq -1 \).

- `PlanarFlow`: A class defining the planar flow transformation.


In [ ]:
class PlanarFlow(nn.Module):
    def __init__(self, z_dim):
        super(PlanarFlow, self).__init__()
        self.u = nn.Parameter(torch.randn(z_dim))
        self.w = nn.Parameter(torch.randn(z_dim))
        self.b = nn.Parameter(torch.randn(1))

    def forward(self, z):
        linear = torch.matmul(z, self.w) + self.b
        u_hat = self.u + (torch.log(1 + torch.exp(torch.matmul(self.w, self.u))) - 1 - torch.matmul(self.w, self.u)) * self.w / torch.norm(self.w, p=2)
        activation = torch.tanh(linear)
        z_next = z + u_hat * activation
        psi = (1 - activation ** 2) * self.w
        log_det_jacobian = torch.log(torch.abs(1 + torch.matmul(psi, u_hat.T)))
        return z_next, log_det_jacobian


## Encoder Network

The encoder network takes the IV grid and additional features as input and outputs the parameters of the initial approximate posterior distribution.

- `Encoder`: A class defining the encoder neural network.


In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, z_dim):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2_mu = nn.Linear(hidden_dim, z_dim)
        self.fc2_logvar = nn.Linear(hidden_dim, z_dim)

    def forward(self, x):
        h = torch.relu(self.fc1(x))
        mu = self.fc2_mu(h)
        logvar = self.fc2_logvar(h)
        return mu, logvar


## Decoder Network

The decoder network takes the latent variable \( z \) and conditioning features (including strike price and time to maturity) to generate a single implied volatility point.

- `Decoder`: A class defining the decoder neural network.


In [ ]:
class Decoder(nn.Module):
    def __init__(self, z_dim, cond_dim, hidden_dim, output_dim):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(z_dim + cond_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, z, cond_features):
        x = torch.cat([z, cond_features], dim=-1)
        h = torch.relu(self.fc1(x))
        iv = self.fc2(h)
        return iv


## Variational Autoencoder (VAE) with Planar Flows

We integrate the encoder, multiple planar flows, and decoder into a VAE. The model uses the encoded input and planar flows to learn a flexible posterior distribution, and the decoder generates the IV points.

- `VAE`: A class defining the variational autoencoder with planar flows.


In [ ]:
class VAE(nn.Module):
    def __init__(self, input_dim, hidden_dim, z_dim, cond_dim, flow_length):
        super(VAE, self).__init__()
        self.encoder = Encoder(input_dim, hidden_dim, z_dim)
        self.flows = nn.ModuleList([PlanarFlow(z_dim) for _ in range(flow_length)])
        self.decoder = Decoder(z_dim, cond_dim, hidden_dim, 1)  # output_dim=1 for single IV point

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x, cond_features):
        mu, logvar = self.encoder(x)
        z_0 = self.reparameterize(mu, logvar)
        z_k = z_0
        log_det_jacobian_sum = 0
        for flow in self.flows:
            z_k, log_det_jacobian = flow(z_k)
            log_det_jacobian_sum += log_det_jacobian
        iv = self.decoder(z_k, cond_features)
        return iv, mu, logvar, log_det_jacobian_sum


## ELBO Loss Function

We define the Evidence Lower Bound (ELBO) loss function. The ELBO combines the reconstruction term (mean squared error between the predicted and actual IV points) and the KL divergence term (measuring the divergence between the approximate posterior and the prior).

- `elbo_loss`: A function calculating the ELBO loss.


In [ ]:
def elbo_loss(reconstructed_iv, actual_iv, mu, logvar, log_det_jacobian_sum):
    # Reconstruction loss (mean squared error)
    recon_loss = nn.MSELoss()(reconstructed_iv, actual_iv)
    
    # KL Divergence
    kl_div = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    
    # ELBO
    elbo = recon_loss + kl_div - log_det_jacobian_sum.sum()
    return elbo


## Training Loop

We implement the training loop to optimize the model parameters. The loop iterates through the data, performs forward and backward passes, and updates the model parameters using the Adam optimizer.

- `train`: A function that trains the VAE model using the provided data loaders.


In [ ]:
import torch.optim as optim

def train(model, data_loader, cond_features_loader, epochs, learning_rate):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    for epoch in range(epochs):
        for batch_idx, (x, iv) in enumerate(data_loader):
            cond_features = next(iter(cond_features_loader))
            optimizer.zero_grad()
            reconstructed_iv, mu, logvar, log_det_jacobian_sum = model(x, cond_features)
            loss = elbo_loss(reconstructed_iv, iv, mu, logvar, log_det_jacobian_sum)
            loss.backward()
            optimizer.step()
            if batch_idx % 10 == 0:
                print(f'Epoch {epoch}, Batch {batch_idx}, Loss: {loss.item()}')

## Full Implementation Example

Below is the full implementation example, combining all the components discussed above. This includes generating synthetic data, defining the model, and training it using the defined functions.

- **Step 1:** Generate synthetic data for IV surfaces and features.
- **Step 2:** Define the Planar Flow layer to ensure invertibility.
- **Step 3:** Create the Encoder and Decoder networks.
- **Step 4:** Implement the VAE with planar flows.
- **Step 5:** Define the ELBO loss function.
- **Step 6:** Train the model using the training loop.


In [ ]:
# Define model parameters
input_dim = encoder_inputs.shape[1]
hidden_dim = 128
z_dim = 32
cond_dim = features_tensor.shape[1] + strike_times_tensor.shape[1]
flow_length = 2

# Instantiate the VAE model
model = VAE(input_dim, hidden_dim, z_dim, cond_dim, flow_length)

In [ ]:
# Training parameters
epochs = 20
learning_rate = 0.001

# Train the model using the defined train function
train(model, data_loader, cond_features_loader, epochs, learning_rate)